# Volcanoes and Emissions

### Building a visualization similar to this app http://volcano.si.axismaps.io/ 

### Imports and function definitions

In [116]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template for Equirectangular Panoramas, plus Volcanoes and Emissions

In [250]:
VEMapTemplate='''mesh
{
    data panorama ./modules/$folderName/sphere.obj
    data vData    ./modules/$folderName/E3WebApp_Eruptions1960.raw
    data eData    ./modules/$folderName/E3WebApp_Emissions.raw
    
    cullRadius $cr
    glslVersion 330


    propertyCollection
    {
        __objectName__
        {
        
            vec4f ColorMultiplier     $ColorMultiplier
            vec3f RotationAxis        $RotationAxis
            vec1f RotationAngle       $RotationAngle
            
            vec1f simBindRealtime 0. #bind the time of this model to the actual time in Uniview (0 == no, 1 == yes)
            vec1f simUseTime 1960. #if simBindReadtime == 0, then this is the time to use (yr)
            vec1f simDtmin 0.1 #minimum time to leave the symbol visible before fading
            vec1f simSize 0.02 #size scale for the symbols
            vec1f simTfade 1 #number of years over which to fade out
            vec1f simAlphaMin 0.1 #minimum opacity after fading out
            vec3f simVolcanoColor 1. 0.15 0. #color for the volcanoes
            vec3f simEmissionColor 1. 1. 0. #color for the emissions

        }

    }


#####################################################
    pass #EarthMap
    {
        passEnable true

        useDataObject panorama
        shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/equirectangularPanorama.vs
                fragmentShader ./modules/$folderName/equirectangularPanorama.fs
                
                texture panorama ./modules/$folderName/$panorama
                {
                }
                

                parameter1f Scale $Scale
                
                stateManagerVar __objectName__.ColorMultiplier      ColorMultiplier
                stateManagerVar __objectName__.RotationAxis         RotationAxis
                stateManagerVar __objectName__.RotationAngle        RotationAngle
                stateManagerVar __objectName__.Alpha                uv_alpha
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    #UV_CULL_MODE GL_FRONT
                    UV_BLEND_ENABLE true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    
#####################################################
    
    pass #Volcanoes
    {
        passEnable true

        useDataObject vData

        shader
        {
            type defaultMeshShader
            {

                parameter1f Scale $Scale
                parameter1f simMarker 1
        
                stateManagerVar __objectName__.RotationAxis         RotationAxis
                stateManagerVar __objectName__.RotationAngle        RotationAngle
                stateManagerVar __objectName__.Alpha                uv_alpha

                StateManagerVar __objectName__.simBindRealtime simBindRealtime
                StateManagerVar __objectName__.simUseTime simUseTime
                StateManagerVar __objectName__.simDtmin simdtmin
                StateManagerVar __objectName__.simSize simSize
                StateManagerVar __objectName__.simTfade simTfade
                StateManagerVar __objectName__.simAlphaMin simAlphaMin
                StateManagerVar __objectName__.simVolcanoColor simColor


                vertexShader ./modules/$folderName/VE_shader.vs
                fragmentShader ./modules/$folderName/VE_shader.fs                
                geometryShader ./modules/$folderName/VE_shader.gs



                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_DEPTH_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_ONE GL_ONE_MINUS_SRC_ALPHA
                    UV_LINE_WIDTH 4

                }
             }
        }
    }

#####################################################
    
    pass #Emissions
    {
        passEnable true

        useDataObject eData

        shader
        {
            type defaultMeshShader
            {

                parameter1f Scale $Scale
                parameter1f simMarker 0
        
                stateManagerVar __objectName__.RotationAxis         RotationAxis
                stateManagerVar __objectName__.RotationAngle        RotationAngle
                stateManagerVar __objectName__.Alpha                uv_alpha

                StateManagerVar __objectName__.simBindRealtime simBindRealtime
                StateManagerVar __objectName__.simUseTime simUseTime
                StateManagerVar __objectName__.simDtmin simdtmin
                StateManagerVar __objectName__.simSize simSize
                StateManagerVar __objectName__.simTfade simTfade
                StateManagerVar __objectName__.simAlphaMin simAlphaMin
                StateManagerVar __objectName__.simEmissionColor simColor


                vertexShader ./modules/$folderName/VE_shader.vs
                fragmentShader ./modules/$folderName/VE_shader.fs                
                geometryShader ./modules/$folderName/VE_shader.gs



                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_DEPTH_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_ONE GL_ONE_MINUS_SRC_ALPHA
                    UV_LINE_WIDTH 4

                }
             }
        }
    }

}

'''

### Volcanoes, Emissions, and Panorama Class

In [251]:
class VEPclass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("sphere.obj",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("equirectangularPanorama.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("equirectangularPanorama.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("VE_shader.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("VE_shader.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("VE_shader.fs",dataFolder)
        self.panorama = "UNINITIALIZED"
        self.ColorMultiplier = "1 1 1 1"
        self.Scale = 1
        self.RotationAxis = "1 0 0"
        self.cr = 1000
        self.RotationAngle = 0
    def setPanorama(self, panorama):
        uvmod.Utility.ensurerelativepathexsists(panorama,dataFolder)
        self.panorama = panorama
    def setColorMultiplier(self,ColorMultiplier):
        self.ColorMultiplier = " ".join(str(c) for c in ColorMultiplier)
    def setScale(self,Scale):
        self.Scale = Scale
    def setRotationAxis(self,RotationAxis):
        self.RotationAxis = " ".join(str(c) for c in RotationAxis)
    def setRotationAngle(self,RotationAngle):
        self.RotationAngle = RotationAngle
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(VEMapTemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 panorama = self.panorama,
                                 ColorMultiplier = self.ColorMultiplier,
                                 Scale = self.Scale,
                                 RotationAxis = self.RotationAxis,
                                 RotationAngle = self.RotationAngle))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [252]:
VEMap = VEPclass(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [264]:
VEMap.object.setcameraradius(10)
VEMap.object.setcoord("SolarSystem")
VEMap.object.setname("VolcanoesEmissions")
VEMap.object.setguiname("/KavliLecture/Schrag/Volcanoes and Emissions")
VEMap.object.settargetradius(20)
VEMap.object.showatstartup(False)
VEMap.setPanorama("BlankMap-World6-Equirectangular.png")
VEMap.setColorMultiplier((1.,1.,1.,1.))
VEMap.setScale(100)
VEMap.cr = 10000


modinfo.setname("Volcanoes and Emissions")
#modinfo.setauthor("Mark SubbaRao<sup>1</sup> and Lisa Kaltenegger<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Cornell University<br/>panorama from:The Reefs of Belize: <br/> A co-production by the Massachusetts Institute of Technology (MIT) and the Charles Hayden Planetarium, Museum of Science")
modinfo.cleardependencies()
#modinfo.adddependency("AlphaCent")
#modinfo.setdesc("A spherical panorama of a coral reef")
#modinfo.setthumbnail("data/TheReefsOfBelize_VR_002064.png")
modinfo.setversion("1.0")

generator.generate("VolcanoesEmissions",[],[VEMap],modinfo)
uvmod.Utility.senduvcommand(VEMap.object.name+".reload")


### Helper Functions for modifing code
##### Reload Module and Shaders in Uniview

In [121]:
uvmod.Utility.senduvcommand(VEMap.object.name+".reload")


Unable to connect to Uniview


Copy modified Shader files and reload

In [122]:
from config import Settings


In [123]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+VEMap.object.name)
uvmod.Utility.senduvcommand(VEMap.object.name+".reload")

Unable to connect to Uniview
